# UserRecommend

La función UsersRecommend tiene como objetivo proporcionar a los usuarios la información sobre los tres juegos más recomendados para un año específico. La función toma como entrada un año y devuelve el top 3 de juegos más recomendados por la comunidad de usuarios. Solo se considerarán comentarios positivos de las reseñas. 

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

### Importación de librerias

In [9]:
import pandas as pd

### Lectura de los CSV

In [10]:
steam_games = pd.read_csv("..\\Datsets\\CSV\\steam_games.csv")
user_reviews =pd.read_csv("..\\Datsets\\CSV\\user_reviews.csv")

### Creación de DataFrame

Para la implementación de esta función, se utilizarán dos DataFrames: user_reviews y steam_games. Estos DataFrames se concatenarán  para obtener las columnas necesarias que serán utilizadas en la función.

In [11]:
# Combinar DataFrames basado en las columnas id y item_id
df_UsersRecommend = pd.merge(user_reviews, steam_games, left_on='item_id', right_on='id', how='inner')
df_UsersRecommend.head()

,user_id,item_id,recommend,posted_year,sentiment_score,sentiment_analysis,genres,app_name,tags,specs,price,early_access,id,developer,release_year
0,76561197970982479,1250,True,2011.0,0.8481,2,"['Action', 'Adventure']",Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...","['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,0.0,1250.0,Tripwire Interactive,2009.0
1,death-hunter,1250,True,2015.0,0.7644,2,"['Action', 'Adventure']",Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...","['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,0.0,1250.0,Tripwire Interactive,2009.0
2,DJKamBer,1250,True,2013.0,-0.5348,0,"['Action', 'Adventure']",Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...","['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,0.0,1250.0,Tripwire Interactive,2009.0
3,diego9031,1250,True,2015.0,0.0000,1,"['Action', 'Adventure']",Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...","['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,0.0,1250.0,Tripwire Interactive,2009.0
4,76561198081962345,1250,True,2014.0,0.0000,1,"['Action', 'Adventure']",Killing Floor,"['FPS', 'Zombies', 'Co-op', 'Survival', 'Actio...","['Single-player', 'Multi-player', 'Co-op', 'Cr...",19.99,0.0,1250.0,Tripwire Interactive,2009.0


Seleccionamos las columnas que van a ser necesarias 

In [12]:
# Seleccionar las columnas necesarias
selected_columns = ['app_name', 'posted_year', 'recommend', 'sentiment_analysis']
df_UsersRecommend = df_UsersRecommend[selected_columns]
df_UsersRecommend.head()

,app_name,posted_year,recommend,sentiment_analysis
0,Killing Floor,2011.0,True,2
1,Killing Floor,2015.0,True,2
2,Killing Floor,2013.0,True,0
3,Killing Floor,2015.0,True,1
4,Killing Floor,2014.0,True,1


Filtrar los comentarios positivos y neutrales

In [13]:
#Filtramos los comentarios positivos y neutrales
df_UsersRecommend =df_UsersRecommend[df_UsersRecommend['sentiment_analysis']!=0]
df_UsersRecommend.head()

,app_name,posted_year,recommend,sentiment_analysis
0,Killing Floor,2011.0,True,2
1,Killing Floor,2015.0,True,2
3,Killing Floor,2015.0,True,1
4,Killing Floor,2014.0,True,1
5,Killing Floor,2010.0,True,2


Agrupar por el nombre del juego y la fecha de la recomendación

In [14]:
#Agrupamos por app_name , posted_year y sumamos la cantidad de recomendaciones
df_UsersRecommend = df_UsersRecommend.groupby(['app_name', 'posted_year']).agg({'recommend': 'sum'}).reset_index()
df_UsersRecommend = df_UsersRecommend.sort_values('recommend',ascending=False)
df_UsersRecommend.head()

,app_name,posted_year,recommend
3232,Team Fortress 2,2014.0,1416
703,Counter-Strike: Global Offensive,2015.0,1317
702,Counter-Strike: Global Offensive,2014.0,949
3231,Team Fortress 2,2013.0,749
1396,Garry's Mod,2014.0,684


### Convertir el DataFrame a CSV

In [15]:
df_UsersRecommend.to_csv('..\\Datsets\\Archivos_API\\UsersRecommend.csv', index=False)


## Función UserRecommend

In [20]:
def UsersRecommend(anio):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    
    Args:
    - anio (int): El año para el cual se desean obtener las recomendaciones.

    Returns:
    - dict: Un diccionario que contiene el top 3 de juegos recomendados en el formato:
        {"Puesto 1": "Nombre del Juego1", "Puesto 2": "Nombre del Juego2", "Puesto 3": "Nombre del Juego3"}
        En caso de no haber recomendaciones para el año especificado, devuelve un mensaje indicando esto.
    """

    # Filtrar las recomendaciones para el año dado y recomendaciones positivas/neutrales
    recomendaciones = df_UsersRecommend[df_UsersRecommend['posted_year'] == anio]

    # Verificar si hay revisiones para el año dado
    if recomendaciones.empty:
        return f"No hay recomendaciones para el año {anio}"

    # Ordenar en orden descendente por la cantidad de recomendaciones
    recomendaciones = recomendaciones.sort_values('recommend', ascending=False)

    # Crear una única línea de resultado
    resultado = {
        "Puesto 1": recomendaciones.iloc[0]['app_name'],
        "Puesto 2": recomendaciones.iloc[1]['app_name'],
        "Puesto 3": recomendaciones.iloc[2]['app_name']
    }

    return resultado


In [21]:
UsersRecommend(2011)

{'Puesto 1': 'Team Fortress 2', 'Puesto 2': 'Portal 2', 'Puesto 3': 'Terraria'}